In [1]:
from pathlib import Path
from drifts_results import compute_counts_from_results, load_analyzed_df, cast_dataset_str
try:
    from IPython import get_ipython
    ip = get_ipython()
    BASE_DIR = Path(ip.run_line_magic('pwd', '')).resolve()
except Exception:
    BASE_DIR = Path.cwd().resolve()
RESULTS_DIR = BASE_DIR / 'results'
FR_CSV = BASE_DIR / 'forest_report.csv'


In [2]:

from pathlib import Path
import json, re, pandas as pd
from drifts_results import compute_counts_from_results, load_analyzed_df, cast_dataset_str, CAT_LIST, DB_TO_CAT

# === Rilevamento automatico dei percorsi (robusto per notebook) ===
try:
    from IPython import get_ipython
    ip = get_ipython()
    BASE_DIR = Path(ip.run_line_magic('pwd', '')).resolve()
except Exception:
    BASE_DIR = Path.cwd().resolve()
RESULTS_DIR = BASE_DIR / "results"
FR_CSV = BASE_DIR / "forest_report.csv"

print("📁 BASE_DIR     :", BASE_DIR)
print("📁 RESULTS_DIR  :", RESULTS_DIR)
print("📄 FOREST_REPORT:", FR_CSV.exists())


📁 BASE_DIR     : C:\Users\danie\Projects\GitHub\drifts
📁 RESULTS_DIR  : C:\Users\danie\Projects\GitHub\drifts\results
📄 FOREST_REPORT: True


In [3]:
from drifts_results import DISPLAY_CATEGORIES, DISPLAY_LABELS, DISPLAY_NAMES

# === Calcolo e merge ===

counts_df = compute_counts_from_results(RESULTS_DIR, verbose=True)
analyzed_df = load_analyzed_df(FR_CSV)

counts_df   = cast_dataset_str(counts_df)
analyzed_df = cast_dataset_str(analyzed_df)

# Diagnostic: inspect zip archives for redis manifests
import zipfile
missing = []
for z in sorted(RESULTS_DIR.glob('*.zip')):
    try:
        with zipfile.ZipFile(z, 'r') as zz:
            names = zz.namelist()
            has = any('redis_backup_db' in n for n in names)
            if not has:
                missing.append((z.name, names[:10]))
    except Exception as e:
        missing.append((z.name, f'error: {e}'))

if missing:
    print('ZIP senza redis manifest (primi entry mostrati):')
    for name, sample in missing:
        print('-', name, '->', sample)
else:
    print('Tutti gli zip contengono redis_backup_db*.json (o non ci sono zip).')

# Restringi ai dataset presenti nei risultati contati
if counts_df.empty:
    print(f"Nessun redis manifest valido trovato in {RESULTS_DIR}")
    results_datasets = []
else:
    results_datasets = sorted(counts_df["dataset"].astype(str).unique())
    print('Dataset conteggiati:', ', '.join(results_datasets))

analyzed_df_res = analyzed_df[analyzed_df["dataset"].isin(results_datasets)].copy()

if results_datasets and not analyzed_df_res.empty:
    merged_results_only = analyzed_df_res.merge(counts_df, on="dataset", how="inner")
else:
    import pandas as pd  # type: ignore
    merged_results_only = pd.DataFrame(columns=["dataset", *DISPLAY_CATEGORIES])

if not merged_results_only.empty:
    for c in DISPLAY_CATEGORIES:
        if c not in merged_results_only.columns:
            merged_results_only[c] = 0
    merged_results_only[DISPLAY_CATEGORIES] = (
        merged_results_only[DISPLAY_CATEGORIES]
        .fillna(0)
        .astype(int)
    )

    merged_results_only["TOT"] = merged_results_only[DISPLAY_CATEGORIES].sum(axis=1)
    merged_results_only = merged_results_only.sort_values("R", ascending=False)

summary_cols = ["dataset", *DISPLAY_CATEGORIES, "TOT"]
summary_cols = [c for c in summary_cols if c in merged_results_only.columns]
summary = merged_results_only[summary_cols].copy()

rename_map = {c: DISPLAY_LABELS.get(c, DISPLAY_NAMES.get(c, c)) for c in DISPLAY_CATEGORIES}
rename_map["TOT"] = DISPLAY_LABELS.get("TOT", "Total")
summary = summary.rename(columns=rename_map)

try:
    _ = style_summary_table
except NameError:
    def style_summary_table(df: pd.DataFrame):
        return (
            df.style.set_properties(**{"text-align": "left"})
              .set_table_styles([{ "selector": "th", "props": [("text-align", "left")] }])
        )

style_summary_table(summary)


?? Errore leggendo ChlorineConcentration_1_false_0/redis_backup_db1.json in C:\Users\danie\Projects\GitHub\drifts\results\ChlorineConcentration_1_false_0.zip
Manifest not found in C:\Users\danie\Projects\GitHub\drifts\results\ElectricDevices_1_false_0.zip
?? Errore leggendo FacesUCR_1_false_0/redis_backup_db1.json in C:\Users\danie\Projects\GitHub\drifts\results\FacesUCR_1_false_0.zip
ZIP senza redis manifest (primi entry mostrati):
- ElectricDevices_1_false_0.zip -> ['ElectricDevices_1_false_0/', 'ElectricDevices_1_false_0/worker_cache_logged_worker_local_workers_7_20251024_024923.log', 'ElectricDevices_1_false_0/worker_cache_logged_worker_local_workers_21_20251023_200518.log', 'ElectricDevices_1_false_0/worker_cache_logged_worker_local_workers_15_20251024_024923.log', 'ElectricDevices_1_false_0/worker_cache_logged_worker_local_workers_29_20251024_024923.log', 'ElectricDevices_1_false_0/worker_cache_logged_worker_local_workers_8_20251023_200518.log', 'ElectricDevices_1_false_0/worker_

,dataset,Candidate,Reason,Non-reason,Candidate Anti-reason,Anti-reason,Good profile,Bad profile,Preferred reason,Anti-reason profile,Total
6,SonyAIBORobotSurface1,0,4902,727,303484,12237,3784,573,0,260,325967
3,HandOutlines,3607,1931,411,50810,555,1865,349,0,359,59887
0,ChlorineConcentration,0,707,0,0,0,32,0,678,0,1417
7,Wafer,637101,633,0,0,0,32,0,606,0,638372
4,MelbournePedestrian,38823,239,0,0,0,32,0,243,0,39337
5,MiddlePhalanxOutlineCorrect,111624,132,8,71352,39,40,8,110,39,183352
2,FacesUCR,0,105,0,0,0,32,0,77,0,214
1,ECG200,117381,43,6,105182,36,38,6,29,36,222757


In [4]:

# === Esporta CSV ===
OUT = BASE_DIR / "analyzed_counts_results_only.csv"
merged_results_only.to_csv(OUT, index=False)
print("✅ Esportato:", OUT)


✅ Esportato: C:\Users\danie\Projects\GitHub\drifts\analyzed_counts_results_only.csv
